In [ ]:
!pip install wandb

In [ ]:
# mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [1]:
!pip install kogpt2_transformers
!pip install rouge

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# 라이브러리 불러오기

import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model, get_kogpt2_tokenizer
import csv
import pandas as pd
from datasets import Dataset
from rouge import Rouge
import re
import os

In [3]:
# 토크나이저 및 모델 선언

tokenizer = get_kogpt2_tokenizer()
model = get_kogpt2_model()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# 스페셜 토큰 적용

special_words = [
            "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
            "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
            "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
            "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
            '#상거래(쇼핑)#', '#개인 및 관계#', '#식음료#', '#시사/교육#', '#미용과 건강#', '#일과 직업#', '#여가 생활#', 
            '#주거와 생활# ', '#행사# ']

tokenizer.add_special_tokens({"additional_special_tokens": special_words}) 

# 임베딩 바꾸기
model.resize_token_embeddings(len(tokenizer))

In [ ]:
text = '#@주소# 여기로 와'
tokenizer.tokenize(text)

In [ ]:
train = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train.tsv', delimiter = '\t')
val = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val.tsv', delimiter = '\t')

In [ ]:
train.head()

In [ ]:
train['Text'].iloc[0]

In [ ]:
# # Category별 DataFrame을 출력

# Category = train['Category'].unique()
# data_path = '/aiffel/aiffel/aiffelthon/csv/'
# def categori_ext_random(data):
#     df = pd.DataFrame()
#     for c in Category:
#         df = pd.concat([df, data[data['Category'] == c].sample(frac = 0.05)], axis = 0)
#         df.reset_index(inplace=True, drop=True)
#         df.to_csv(data_path+'train_5%_random.tsv', sep = '\t', encoding = 'utf-8', index = False)
#     return df

In [ ]:
# # Category별 DataFrame을 출력

# Category = val['Category'].unique()
# data_path = '/aiffel/aiffel/aiffelthon/csv/'
# def val_categori_ext__random(data):
#     df = pd.DataFrame()
#     for c in Category:
#         df = pd.concat([df, data[data['Category'] == c].sample(frac = 0.05)], axis = 0)
#         df.reset_index(inplace=True, drop=True)
#         df.to_csv(data_path+'val_5%_random.tsv', sep = '\t', encoding = 'utf-8', index = False)
#     return df

In [ ]:
# categori_ext_random(train)
# val_categori_ext__random(val)

In [ ]:
# Category별 DataFrame을 출력 (train)
train = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train.tsv', delimiter = '\t')
Category = train['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def train_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'train_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

train_categori_ext(train)

In [ ]:
# Category별 DataFrame을 출력
val = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val.tsv', delimiter = '\t')
Category = val['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def val_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'val_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

val_categori_ext(val)

In [ ]:
# torch cash 초기화

import gc
gc.collect()
torch.cuda.empty_cache()

Model : KoGPT-2

Parametors size : 

[How to use GPT2LMHeadModel for conditional generation](https://github.com/huggingface/transformers/issues/970)

Data NAME : Ai_hub 대화문 요약 Data Set

In [4]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ.,?!]+[/ㄱ-ㅎㅏ-ㅣ.,?!]', '', sentence) # 여러개 자음, 모음, 구두점에서 마지막 1개만 남긴다
    sentence = re.sub("[^가-힣a-z0-9-.!?@#]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [5]:
# 데이터 로드 및 전처리와 메타데이터 적용 

def data_load(data_path = '/aiffel/aiffel/aiffelthon/csv/train_5%.tsv'):
    data = []
    train_df = pd.read_csv(data_path, delimiter = '\t')
    #train_df = train_df.iloc[::2, :] # 짝수 row data만 추출한다.
    #train_df.reset_index(drop=True, inplace = True) # 인덱스 초기화
    
    # 전처리 적용
    clean_text = []
    
    for s in tqdm(train_df['Text']):
        clean_text.append(preprocess_sentence(s))
                          
    train_df['Text'] = clean_text
    
    clean_headlines = []
                          
    for s in tqdm(train_df['Summary']):
        clean_headlines.append(preprocess_sentence(s))
        
    train_df['Summary'] = clean_headlines 
    
    # 메타데이터 적용
#     def metadata_train(train_df): 
#         meta = '#'+train_df['Category']+'# '
#         train_df['Text'] = meta + train_df['Text']
#         return train_df['Text']
    
#     metadata_train(train_df)
    
    # Series > list
    train_df['Text'].tolist()
    train_df['Summary'].tolist()
    
    return train_df['Text'], train_df['Summary']

def val_load(data_path = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv'):
    val_df = pd.read_csv(data_path, delimiter = '\t')
    #val_df = val_df.iloc[::2, :]
    #val_df.reset_index(drop=True, inplace = True)
    

    # 전처리 적용
    clean_text_val = []
                          
    for s in tqdm(val_df['Text']):
        clean_text_val.append(preprocess_sentence(s))
    
    val_df['Text'] = clean_text_val
    
    clean_headlines_val = []
                          
    for s in tqdm(val_df['Summary']):
        clean_headlines_val.append(preprocess_sentence(s))
        
    val_df['Summary'] = clean_headlines_val 
    
    # 메타데이터 적용
#     def metadata_val(val_df): 
#         meta = '#'+val_df['Category']+'# '
#         val_df['Text'] = meta + val_df['Text']
#         return val_df['Text']
    
#     metadata_val(val_df)
                          
    # Series > list
    val_df['Text'].tolist()
    val_df['Summary'].tolist()
    
    return val_df['Text'], val_df['Summary']

In [ ]:
data_1 = data_load()

In [ ]:
data_1

In [ ]:
data_1

In [ ]:
data_2 = val_load()

In [ ]:
type(data_1)

In [ ]:
data_2

### hyparam은 Code를 가져올 때 설정을 유지중입니다. GPT-2  학습이 제대로 이루어진게 어제부터라서 수정하지는 못했습니다.

### 그래도 수정을 한 부분은 문장을 생성할 때 eos_token, pad_token이 생성되어 이 부분을 출력하지 않도록 bad_words_index를 사용했습니다.

In [6]:
# Model 제작 - beam serach

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model


class AbstractiveKoGPT2(nn.Module):
    def __init__(self):
        super(AbstractiveKoGPT2, self).__init__()
        self.kogpt2 = get_kogpt2_model()

    def generate(self,
                 input_ids,
                 #do_sample=True,
                 max_length= 128,
                 num_beams=5,
                 length_penalty = 1.2,
                 #top_p=0.92, # top_p, k는 code를 가져올 때 설정을 유지하였고 보내주신 텍스트 생성
                 #top_p=50,
                 temperature= 0.6, # 0.6 or 0.8 
                 no_repeat_ngram_size = None, # 변경 시도해보기
                 num_return_sequences=3,
                 early_stopping=False,
                 bad_words_ids = [1, 3]
                 ):
        return self.kogpt2.generate(input_ids,
                                     #do_sample = do_sample,
                                     max_length = max_length,
                                     num_beams = num_beams, # 더 많은 글 작성 예시 생성
                                     length_penalty = length_penalty,
                                     #top_p = top_p, # 글의 표현 범위 조절
                                     #top_k = top_k, # 글의 표현 범위 조절
                                     temperature = temperature, # 글의 창의성 조절
                                     no_repeat_ngram_size= no_repeat_ngram_size,
                                     num_return_sequences = num_return_sequences,
                                     early_stopping = early_stopping,
                                     eos_token_id = 1,
                                     pad_token_id = 3,
                                    )

    def forward(self, input, labels = None):
        if labels is not None:
            outputs = self.kogpt2(input, labels=labels)
        else:
            outputs = self.kogpt2(input)

        return outputs

In [ ]:
# Model 제작 - 샘플링

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model


class AbstractiveKoGPT2(nn.Module):
    def __init__(self):
        super(AbstractiveKoGPT2, self).__init__()
        self.kogpt2 = get_kogpt2_model()

    def generate(self,
                 input_ids,
                 do_sample=True,
                 max_length= 128,
                 #num_beams=5,
                 top_p=0.9, # top_p, k는 code를 가져올 때 설정을 유지하였고 보내주신 텍스트 생성
                 top_k=50,
                 temperature= 0.9, # 0.6 or 0.8 
                 no_repeat_ngram_size = None, # 변경 시도해보기
                 num_return_sequences=3,
                 early_stopping=False,
                 bad_words_ids = [1, 3]
                 ):
        return self.kogpt2.generate(input_ids,
                                     do_sample = do_sample,
                                     max_length = max_length,
                                     #num_beams = num_beams, # 더 많은 글 작성 예시 생성
                                     top_p = top_p, # 글의 표현 범위 조절
                                     top_k = top_k, # 글의 표현 범위 조절
                                     temperature = temperature, # 글의 창의성 조절
                                     no_repeat_ngram_size= no_repeat_ngram_size,
                                     num_return_sequences = num_return_sequences,
                                     early_stopping = early_stopping,
                                     eos_token_id = 1,
                                     pad_token_id = 3,
                                    )

    def forward(self, input, labels = None):
        if labels is not None:
            outputs = self.kogpt2(input, labels=labels)
        else:
            outputs = self.kogpt2(input)

        return outputs

In [ ]:
# text = '#주거와 생활# 안녕! 난 기락이야~'

# tok_text = tokenizer.tokenize(text, add_special_tokens=False)
# print(tok_text)
# input_ids = tokenizer.encode(text, add_special_tokens=False)
# print(input_ids)
# decoded_ids = tokenizer.decode(input_ids, skip_special_tokens=True)
# print(decoded_ids)

### 스페셜 토큰은 당시 대회 1등을 수상한 팀이 사전에 파악해둔 라벨링 텍스트를 토대로 가져왔으며, 정규표현식을 사용해 해당 텍스트가 데이터에 포함된 것을 확인했습니다.

In [7]:
# 학습용 데이터 세트 제작

class AbstrativeDataset(Dataset):
    def __init__(self,
                 #n_ctx = 1024, 
                 n_ctx = 256, 
                 texts_max_length = 128,
                 summary_max_length = 64,
                 device = 'cpu'
                 ):
        
        self.data =[]
        self.device = device
        self.tokenizer = get_kogpt2_tokenizer()

        bos_token_id = [self.tokenizer.bos_token_id] # , 0
        eos_token_id = [self.tokenizer.eos_token_id] # , 1
        pad_token_id = [self.tokenizer.pad_token_id] # , 3
                
        #self.
        # 스페셜 토큰은 적용

        datas = data_load()
        
        for texts, text_sum in tqdm(zip(datas[0],datas[1])):
            text = texts
            abstractive_summary = text_sum

#             tmp_str =''
#             for article in articles:
#                 tmp_str += article

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(text, truncation= True, max_length=texts_max_length, add_special_tokens=False)
            enc_abstractive_summary = self.tokenizer.encode(abstractive_summary, truncation= True, max_length=summary_max_length)
            
            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id + enc_abstractive_summary + eos_token_id
            pad_token_len = n_ctx - len(index_of_words)
            index_of_words += pad_token_id * pad_token_len
            #print(f'max: {max(index_of_words)}, min: {min(index_of_words)}')
            
            # RuntimeError: CUDA error: device-side assert triggered
            if max(index_of_words)<50156:
                self.data.append(torch.tensor(index_of_words).to(device))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

In [ ]:
data_3 = AbstrativeDataset()

In [ ]:
data_3

In [ ]:
# 데이터 셋이 제대로 만들어졌는지 확인 완료

len(data_3)

In [ ]:
# 데이터 셋이 제대로 만들어지지 않은 경우

len(dataset)

In [8]:
# 검증용 데이터 세트 제작
class EvalAbstrativeDataset(Dataset):
    def __init__(self,
                 device,
                 tokenizer,
                 #n_ctx = 1024,
                 n_ctx = 256,
                 data_path = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv',
                 texts_max_length = 128,
                 summary_max_length = 64,
                 ):
        #self.data = pd.DataFrame()
        self.data =[]
        self.tokenizer = tokenizer
        print(device)
        
        bos_token_id = [self.tokenizer.bos_token_id] # 
        eos_token_id = [self.tokenizer.eos_token_id] # 

        #json_datas = json_load(data_path=data_path)
        #eval_datas = data_load(data_path=data_path)

#         special_words = [
#             "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
#             "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
#             "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
#             "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
#             '#상거래(쇼핑)# ',
#             '#개인 및 관계# ', '#식음료# ', '#시사/교육# ', '#미용과 건강# ', '#일과 직업# ', '#여가 생활# ', 
#             '#주거와 생활# ', '#행사# ']
        
#         self.tokenizer.add_special_tokens({"additional_special_tokens": special_words})
        
        datas = val_load()
        
        for data in tqdm(datas[0]):
            #id = dict_data['id']
            texts = data

            tmp_str =''
            for text in texts:
                tmp_str += text

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(tmp_str, truncation = True, max_length=texts_max_length, add_special_tokens=False)

            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id

            self.data.append({
                #'id':id,
                'input':torch.tensor([index_of_words]).to(device)
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

### def_seed부분은 아직 이해가 부족하여 수정은 하지않았습니다.

### 현재 1 epoch로 모든 train data(29만개)를 사용할 시 약 5시간이 걸립니다. 데이터를 분할하여 다양한 방법을 적용해보는 것을 목표로하고 성능 향상이 보인다면 전체 데이터를 다 사용할 예정입니다.

### 데이터 분할 기준은 각 Category에서 20%씩 추출하여 사용해볼 생각입니다.

In [9]:
# train

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from torch.utils.data import dataloader
import random
#import wandb

def set_seed(seed = 42): # 난수 생성 seed 고정
    torch.backends.cudnn.deterministic = True # Deterministic 한 알고리즘만 사용하게 하기
    torch.backends.cudnn.benchmark = False # cudnn.benchmar 활성화 여부
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
        
if __name__ == '__main__':
    
    set_seed()
    #wandb.init(project= 'bakbak', entity= 'quarter2000', name= f'KG')
    #checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_2/"
    #save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_2.pth"
    checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_2"
    save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_1epoch_metax_beam.pth"
    
    n_epoch = 1         # Num of Epoch
    batch_size = 4      # 배치 사이즈
    #batch_size =       # 배치 사이즈
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #device = 'cpu'
    print('device: ',device)
    save_step = 200 # 학습 저장 주기
    learning_rate = 5e-4  # Learning Rate

    dataset= AbstrativeDataset(device=device) 
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    #checkpoint = torch.load(save_ckpt_path,map_location=torch.device(device))
    model = AbstractiveKoGPT2()
    model.to(device)
    #model.load_state_dict(checkpoint['model_state_dict'])

    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_losses = []
    losses =[]

    if os.path.isfile(save_ckpt_path): # 체크포인트가 존재한다면 불러오기
        checkpoint = torch.load(save_ckpt_path, map_location=device)
        pre_epoch = checkpoint['epoch']
        pre_loss = checkpoint['loss']
        total_losses = checkpoint['losses']

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}, loss={pre_loss}")
        

    for epoch in tqdm(range(n_epoch)):
        count = 0
        with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
            for i, data in enumerate(train_loader):
                optimizer.zero_grad()
#                 for d in data:
#                     print(max(d),min(d))
                outputs = model(data, labels=data)
                _, logits = outputs[:2]

                # Shift so that tokens < n predict n
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = data[..., 1:].contiguous()

                loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
                loss.backward()
                optimizer.step()

                losses.append(loss.item())

                if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                    torch.save({
                        'epoch': epoch,
                        'train_no': count,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss,
                        'losses': losses
                    }, save_ckpt_path)
                count += 1
                pbar.update(1)
                pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")
                #if i%50==0:
                    #wandb.log({'loss':np.mean(losses)})

        total_losses.append(np.mean(losses))

#     # data
    t_data = {
        "loss": total_losses
    }
    df = pd.DataFrame(t_data)
    display(df)

#     # graph
#     plt.figure(figsize=[12, 4])
#     plt.plot(losses, label="loss")
#     plt.legend()
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.show()

device:  cuda


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
100%|██████████| 13994/13994 [00:00<00:00, 86357.96it/s]
13994it [00:09, 1431.19it/s]
Train(0):   1%|▏         | 46/3499 [00:13<16:34,  3.47it/s, Loss: 4.838 (5.288)]


Train(0):   3%|▎         | 94/3499 [00:27<16:35,  3.42it/s, Loss: 5.206 (5.147)]


Train(0):   4%|▍         | 142/3499 [00:41<16:31,  3.38it/s, Loss: 4.740 (5.072)]


Train(0):   5%|▌         | 190/3499 [00:55<16:34,  3.33it/s, Loss: 4.341 (5.031)]


...

Train(0):  40%|███▉      | 1386/3499 [08:12<10:42,  3.29it/s, Loss: 4.466 (4.814)]


Train(0):  41%|████      | 1434/3499 [08:40<10:19,  3.33it/s, Loss: 4.444 (4.811)]


Train(0):  42%|████▏     | 1481/3499 [08:54<10:13,  3.29it/s, Loss: 4.577 (4.807)]


  0%|          | 0/1 [09:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# test

tokenizer = get_kogpt2_tokenizer()
ckpt_path = '/aiffel/aiffel/aiffelthon/checkpoint_2/kogpt2-abstractive_1epoch_metaX_beam.pth'
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(device)

eval_datas = EvalAbstrativeDataset(tokenizer=tokenizer,device=device,data_path='/aiffel/aiffel/aiffelthon/csv/val_5%.tsv')

checkpoint = torch.load(ckpt_path,map_location=torch.device(device))
model = AbstractiveKoGPT2()
model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

my_summaries=[]
# for data in tqdm(eval_datas[:3337]):
#     input_ids = data['input']
#     sample_output = model.generate(input_ids=input_ids,max_length=1024)
#     summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1])
#     summary = summary.replace('','').replace('','')
#     my_summaries.append(summary)

for data in tqdm(eval_datas):
    input_ids = data['input']
    max_toc = max(input_ids[0])
    min_toc = min(input_ids[0])
    if max_toc<50156:
        sample_output = model.generate(input_ids=input_ids, max_length=256)
        summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1], skip_special_tokens=True) 
        # 스페셜 토큰은 출력되지 않게 제한을 걸었습니다.
        summary = summary.replace('','').replace('','')
        my_summaries.append(summary)
    else:
        summary = tokenizer.decode(input_ids[0])
        summary = summary.replace('','').replace('','').replace('','')
        my_summaries.append(summary)

In [ ]:
# 요약 데이터 형태 확인

type(my_summaries)

In [ ]:
# 실제 원문과 요약문 불러오기

Input_Text = val_load()[0]
real_Sum = val_load()[1]

In [ ]:
Input_Text[6]

In [ ]:
# 2 에포크 결과

for num in range(0,10):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [ ]:
# 6 에포크 결과

for num in range(0, 5):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [ ]:
# 6 에포크 결과

for num in range(0, 5):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

In [ ]:
# 실제 요약문 Reference로 불러오기

Reference = val_load()[1]


### rouge.get_scores 사용 방법

- rouge.get_scores(Model이 요약한 문장, 실제 요약문, avg = True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 스페셜 토큰 미사용
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 3epoch
# 메타 데이터 삭제
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 3epoch
# 메타 데이터 삭제
# num_beams = 7 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 3epoch
# 메타 데이터 삭제
# num_beams = 3 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 5epoch
# 메타 데이터 삭제
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 삭제
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 삭제
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건 랜덤 추출에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 적용 (스페셜 토큰에 포함)
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5% 추출에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 적용 (스페셜 토큰에 포함) 임베딩 사이즈 변경 적용
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5% 추출에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 적용 (스페셜 토큰에 포함)
# num_beams = 5 적용
# num_return_sequences = 3 적용
# length_penalty = 1.2
# 검증용 데이터 5% 추출에 대한 Rooge-Score의 평균

# length_penalty를 적용하던 안하던 크게 상관이없다.

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 1epoch
# 메타 데이터 적용 (스페셜 토큰에 포함)
# top p = 0.9
# top k = 50
# temperture = 0.9
# num_return_sequences = 3 적용
# 검증용 데이터 5%에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)

In [ ]:
# Rouge-Score 출력 - 3epoch
# 메타 데이터 삭제
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건 랜덤 추출에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
rouge.get_scores(model_out, reference, avg=True)